In [1]:
#Numba#

In [1]:
from numba import njit,jit
import numpy as np
import cv2
import itertools

cap = cv2.VideoCapture('bird.avi')

quesize = 10
stabilityqueue = np.zeros((480,640,quesize,3),dtype=np.int32)
w = np.zeros((480,640),dtype=np.int32)
ID = np.zeros((480,640),dtype=np.int32)
queueLimits = np.zeros((480,640,2),dtype=np.int32)
#print(stabilityqueue[:,:,0,:].shape)

#@njit
def IsFoundinStabilityQueue(pixel,i,j):
    for queueIndex in range(queueLimits[i,j,0],queueLimits[i,j,1]):
        queuevalue = np.reshape(stabilityqueue[i,j,queueIndex,:],(3,1))
        if queuevalue[0][0]==pixel[0][0] and queuevalue[1][0]==pixel[1][0] and queuevalue[2][0]==pixel[2][0]:
            return True
    return False
    
#@njit
def bewis(frame):
    for i in range(frame.shape[0]):
        for j in range(frame.shape[1]):
            r,g,b = frame[i,j,:]
            #create binary endoder for each pixel
            a = np.zeros((256,3))
            a[0:r,0] = 1
            a[0:g,1] = 1
            a[0:b,2] = 1
            #check if pixel color is in front of stability queue
            front = np.reshape(stabilityqueue[i,j,queueLimits[i,j,0],:],(3,1)) 
            pixel = np.reshape(frame[i,j,:],(3,1))
            if front[0][0]==pixel[0][0] and front[1][0]==pixel[1][0] and front[2][0]==pixel[2][0]:
                w[i][j] += ID[i][j]
                ID[i,j] += 1
            elif IsFoundinStabilityQueue(pixel,i,j):
                #move it to front of queue
                ID[i,j] = 1
            else:
                x=0 
                #insert it in front of the queue(with w = 0 abnd id = 1)
                #remove last element of queue
                
            #if pixel color in front of the queue has w > k:
                #use binary pattern to train wisardrp by reward and punishment
                
                
                
                        
while(True):
    ret, frame = cap.read()
    # Display the resulting frame
    cv2.imshow('frame',frame)
    #print(frame.shape)
    bewis(frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()


'''
    print(frame.shape)
    for p in itertools.product(range(frame.shape[0]),range(frame.shape[1])):
        print (p)
        #r,g,b = frame[p[0],p[1],:]
        #a = np.zeros((256,3))
        #a[0:r,0] = 1  
        #a[0:g,1] = 1
        #a[0:b,2] = 1
'''

'\n    print(frame.shape)\n    for p in itertools.product(range(frame.shape[0]),range(frame.shape[1])):\n        print (p)\n        #r,g,b = frame[p[0],p[1],:]\n        #a = np.zeros((256,3))\n        #a[0:r,0] = 1  \n        #a[0:g,1] = 1\n        #a[0:b,2] = 1\n'

In [92]:
import time
from numba import njit,prange,vectorize,int32,float64,int64
import numpy as np
import random
import tensorflow as tf

In [7]:
def preprocessing(tx_train, ty_train, tx_test, ty_test):
        
    py_train = ty_train.flatten()
    py_test = ty_test.flatten()
    
    px_test = tx_test.reshape(10000,input_size)
    px_train = tx_train.reshape(60000,input_size)
    
    px_train = np.asarray(px_train).astype(np.int32)
    py_train = np.asarray(py_train).astype(np.int32)
    px_test = np.asarray(px_test).astype(np.int32)
    py_test = np.asarray(py_test).astype(np.int32)

    return px_train, py_train, px_test, py_test   


In [8]:
class WiSARDrp:                                                     
    
    def __init__(self,input_size,no_of_rand_pix_selec,nodes,ram_address_count,dis_number):
        self.input_size = input_size
        self.no_of_rand_pix_selec = no_of_rand_pix_selec
        self.nodes = nodes
        self.ram_address_count = ram_address_count
        self.dis_number = dis_number 


    def discriminator(self):
        discriminator = []
        accumulated_pos = []
        my_list = list(range(0,self.input_size))
        for i in range(self.dis_number):  #10
            ram = []
            random.shuffle(my_list)
            for j in range((int)((self.nodes))): #98    
                total_pos = []            
                positions = []
                positions = my_list[j*no_of_rand_pix_selec:j*no_of_rand_pix_selec+no_of_rand_pix_selec]
                accumulated_pos.append(positions)
                total_pos = np.vstack(positions)
                #print(total_pos)
                table = []
                dictionary = {}
                
                max = len("{0:b}".format(2**len(total_pos))) - 1
                for i in range(2**len(total_pos)):
                    x = (('0' * max) + "{0:b}".format(i))
                    x = x[len(x)-max:]
                    dictionary[x] = 0
                table.append(dictionary)
                #print(table)
                ram.append(table)
        
            di = []
            for j in range(len(ram)):
                for i in range(len(ram[j])):
                    for key, value in ram[j][i].items():
                        temp = [key,value]
                        di.append(temp)
                        
            discriminator.append(di)
    
        discriminator = np.asarray(discriminator).astype(np.int32)
        accumulated_pos = np.asarray(accumulated_pos).astype(np.int32)
        return discriminator, accumulated_pos
    
                        
    
    @staticmethod
    @njit(parallel = True)
    def train_with_bleeching(d,pos,x_train, y_train):
        
        images = x_train
        lable = y_train    
        
        for i in range(len(images)):
            image = images[i]
            num = lable[i]
            all_ram_of_selected_discriminator = d[num]
            t_ratina = pos[(int)(nodes*num):(int)(nodes*num+nodes)]
            
            for i in range((int)(nodes)):
                part = all_ram_of_selected_discriminator[(ram_address_count*i):(ram_address_count*i+ram_address_count)]
                ratina_for_one_ram = t_ratina[i]
                
                #threshold = 0               
                n = []                                                                
                for ix in range(len(ratina_for_one_ram)):
                    pix = ratina_for_one_ram[ix]
                    if image[(pix-1)]>=1:
                        n.append(1)
                        #threshold = threshold + 1     
                    else:
                        n.append(0)
                
                num = 0
                for i in range(no_of_rand_pix_selec):
                    num = (n[i])*(10**((no_of_rand_pix_selec-1)-i)) + num
            
                address_of_that_ram = (int)(num)
                for key in range(ram_address_count):
                    index = part[key]
                    #if threshold >= 1:          
                    if index[0] == address_of_that_ram:
                        index[1] += 1
                    else:
                        if index[1] != 0:
                            index[1] -= 1



    #@vectorize(['int32(int32,int32,int32,int32)'], target = 'cuda')
    @staticmethod
    @njit(parallel = True)
    def test_with_bleaching(d,pos,x_test,y_test):
        right = 0
        wrong = 0
        images = x_test
        lable = y_test
        ct = 0.02
        b=1
        
        for i in range(len(images)):
            image = images[i]
            actual_lable = lable[i]
            
            total_sum=[]
            
            for ix in range(dis_number):
                
                t_ratina = pos[int((nodes*ix)):(int)((nodes*ix+nodes))]
                
                sum_of_ram_output = 0
                dis = d[ix]
                
                for i in range(int(nodes)):
                    part = dis[(ram_address_count*i):(ram_address_count*i+ram_address_count)]
                    ratina_for_one_ram = t_ratina[i]
                    
                    n = []                                                                
                    for pix in ratina_for_one_ram:
                        if image[(pix-1)]>=1:
                            n.append(1)
                        else:
                            n.append(0)
                    
                    num = 0
                    for i in range(no_of_rand_pix_selec):
                        num = (n[i])*(10**((no_of_rand_pix_selec-1)-i)) + num
                    
                    address_of_that_ram = (int)(num)
                
                    for key in range(len(part)):
                        prt = part[key]
                        if prt[0] == address_of_that_ram and prt[1]>=b:           #have to change this
                            sum_of_ram_output += 1
                
                total_sum.append(sum_of_ram_output)        
        
            max_sum = 0
            sec_max = 0
            idx = 0
            
            for i in range(len(total_sum)):
                if max_sum < total_sum[i]:
                    max_sum = total_sum[i]
                    idx = i
                    
            for j in range(len(total_sum)):
                if sec_max < total_sum[j] and j!=idx:
                    sec_max = total_sum[j]
                    
            index_of_dis = idx
            if index_of_dis == actual_lable:
                right += 1
            else:
                wrong += 1
            
            if max_sum == sec_max:
                confidence = 0
            else:
                confidence = 1 - float(sec_max)/float(max_sum)
                if confidence < ct:
                    b += 1
        
        return right,wrong

In [9]:
input_size = 28*28
no_of_rand_pix_selec = 2**(3)     ## ** (must) no_of_rand_pix_selec = 2^(n) where n is 0,1,2...
nodes = int(input_size/no_of_rand_pix_selec)    #98
ram_address_count = 2**(no_of_rand_pix_selec)   #256
dis_number = 10                #10 i.e number of lables

In [10]:
(tx_train, ty_train), (tx_test, ty_test) = tf.keras.datasets.mnist.load_data()
px_train, py_train, px_test, py_test = preprocessing(tx_train, ty_train, tx_test, ty_test)

In [11]:
w = WiSARDrp(input_size,no_of_rand_pix_selec,nodes,ram_address_count,dis_number)
d, acc_pos = w.discriminator()
print(d.shape)

(10, 25088, 2)


In [12]:
starttrain = time.time()
w.train_with_bleeching(d,acc_pos,px_train[0:60000],py_train[0:60000])
endtrain = time.time()
print("time train = ",endtrain - starttrain)

time train =  18.97050166130066


In [13]:
starttest = time.time()
right,wrong = w.test_with_bleaching(d,acc_pos,px_test[0:10000],py_test[0:10000])
endtest = time.time()
print("time test = ",endtest - starttest)

time test =  31.408536434173584


In [14]:
print("number of right result = ",right)
print("number of wrong results = ",wrong)
accuracy = ((right)/(right+wrong))*100
print("accuracy by testing the model =",accuracy)
#    end = time.time()
#    print("total time = ",end - start)

number of right result =  6037
number of wrong results =  3963
accuracy by testing the model = 60.370000000000005


In [178]:
stabilityqueue = np.zeros((10,10,10,3),dtype=np.int32)

In [179]:
a = np.zeros((10),dtype=np.int32)

In [180]:
for i in range(10):
    a[i] = i

In [181]:
a

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)

In [183]:
a1 = [5]
c = a1 + a
c

array([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

In [175]:
front = a[0]
print(front)

0


In [162]:
def roll(tem):
    c = np.roll(tem,1)
    return(c)

In [156]:
print(roll(a))

[9 0 1 2 3 4 5 6 7 8]


In [ ]:
for i in range(10):
    if a[i] == 6 :  
        roll(a)
        a[0] = 6

In [12]:
import numpy as np
x = np.array([10, 20, 30, 40, 50, 60, 70, 80, 90, 100])
print(x)

#@njit     it wont work in njit
def ad():
    for i in range(len(x)):
        if x[i] == 40:
            new_x = np.delete(x, i)
            new_y = np.insert(new_x,0,40)
            print(new_y)
            
ad()

[ 10  20  30  40  50  60  70  80  90 100]
[ 40  10  20  30  50  60  70  80  90 100]


In [15]:
@njit
def delete_workaround(arr, num):
    mask = np.zeros(arr.shape[0], dtype=np.int64) == 0
    mask[np.where(arr == num)[0]] = False
    return arr[mask]

a = np.array([[1,2,3,4],[5,6,7,8]])

delete_workaround(a, 3)

array([[5, 6, 7, 8]])

In [31]:
x = np.array([9,1,2])
@njit
def delete_workaround(arr):
    b = np.roll(a,3)
    b[0] = x
    return b

a = np.zeros((4,3),dtype=np.int32)

print(delete_workaround(a))

[[9 1 2]
 [0 0 0]
 [0 0 0]
 [0 0 0]]


In [98]:
x = np.array([2,3,4], dtype=np.int64)
y = np.array([4,3,2], dtype=np.int64)
#@vectorize([int64(int64, int64)])
@njit
def delete_workaround(arr,x):
    for i in range(len(a)): 
        if a[i][0] == x[0] and a[i][1] == x[1] and a[i][2] == x[2]:
            print(a[:i+1])
            b = np.roll(a[:i+1],3)
            print(b)
            b[0]=y
            print(c[:i+1])
            print(b)
            c[:i+1] = b
            #print(c)
            c[i+2:] = a[i+2:]
            #print(c)
    return c        

a = np.zeros((4,3),dtype=np.int64)
b = np.zeros((4,3),dtype=np.int64)
c = np.zeros((4,3),dtype=np.int64)
a[1]=[1,2,3]
a[2]=[2,3,4]
print(a.dtype)
print(x.dtype)
#print(a[0][0], a[0][1], len(a))
print(delete_workaround(a,x))

int64
int64


TypingError: Failed in nopython mode pipeline (step: nopython frontend)
Invalid use of Function(<built-in function setitem>) with argument(s) of type(s): (readonly array(int64, 2d, C), slice<a:b>, array(int64, 2d, C))
 * parameterized
In definition 0:
    All templates rejected with literals.
In definition 1:
    All templates rejected without literals.
In definition 2:
    All templates rejected with literals.
In definition 3:
    All templates rejected without literals.
In definition 4:
    TypeError: Cannot modify value of type readonly array(int64, 2d, C)
    raised from /home/iss/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/site-packages/numba/typing/arraydecl.py:179
In definition 5:
    TypeError: Cannot modify value of type readonly array(int64, 2d, C)
    raised from /home/iss/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/site-packages/numba/typing/arraydecl.py:179
In definition 6:
    All templates rejected with literals.
In definition 7:
    All templates rejected without 

In [96]:
x = np.array([2,3,4], dtype=np.int64)
y = np.array([4,3,2], dtype=np.int64)
@vectorize([int64(int64, int64)])
def delete_workaround(arr,x): 
    print(arr,x)
    for i in range(10):
         if a[i-2] == x[i-2] and a == x and a == x:
#         print(a[:i+1])
#         b = np.roll(a[:i+1],3)
#         print(b)
#         b[0]=y
#         print(c[:i+1])
#         print(b)
#         c[:i+1] = b
#         #print(c)
#         c[i+2:] = a[i+2:]
#         #print(c)
    return x        

a = np.zeros((4,3),dtype=np.int64)
b = np.zeros((4,3),dtype=np.int64)
c = np.zeros((4,3),dtype=np.int64)
a[1]=[1,2,3]
a[2]=[2,3,4]
print(a.dtype)
print(x.dtype)
#print(a[0][0], a[0][1], len(a))
print(delete_workaround(a,x))

int64
int64
0 2
0 3
0 4
1 2
2 3
3 4
2 2
3 3
4 4
0 2
0 3
0 4
[[2 3 4]
 [2 3 4]
 [2 3 4]
 [2 3 4]]


In [148]:
def cyclicRotate(input): 
       
     # slice list in two parts and append 
     # last element in front of the sliced list 
       
     # [input[-1]] --> converts last element pf array into list 
     # to append in front of sliced list 
  
     # input[0:-1] --> list of elements except last element 
     print ([input[-1:]] + input[0:-1]) 

In [149]:
cyclicRotate(a)

[[ 9 10 11 12 13 14 15 16 17]]


In [176]:
#Rotate the given array by n times toward left
def ror(arr,n):
    for i in range(0, n):    
    #Stores the first element of the array    
        first = arr[0];    
            
        for j in range(0, len(arr)-1):    
        #Shift element of array by one    
            arr[j] = arr[j+1]
            
        #First element of array will be added to the end    
        arr[len(arr)-1] = first
    return(arr)    

In [177]:
print(ror(a,1))

[1 2 3 4 5 6 7 8 9 0]


In [123]:
def rorm(test_list):
    test_list = test_list[-1:] + test_list[:-1]
    return(test_list)

In [124]:
print(rorm(a))

[ 9 10 11 12 13 14 15 16 17]


In [80]:
for i in range(10):
    if a[i] == 6 :
        print(a)
        roll(a)
        print(a)
        a[0] = 6

[0 1 2 3 4 5 6 7 8 9]
[0 1 2 3 4 5 6 7 8 9]


In [81]:
a

array([6, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)

In [102]:
test_list = [1, 4, 5, 6, 7, 8, 9, 12] 
  
# printing the original list 
print ("The original list is : " + str(test_list)) 
  
# using list slicing and "+" operator 
# shift last element to first 
test_list = test_list[-1:] + test_list[:-1]  
  
# printing result 
print ("The list after shift is : " + str(test_list)) 

The original list is : [1, 4, 5, 6, 7, 8, 9, 12]
The list after shift is : [12, 1, 4, 5, 6, 7, 8, 9]


In [111]:
def rightRotate(lists, num): 
    output_list = [] 
      
    # Will add values from n to the new list 
    for item in range(len(lists) - num, len(lists)): 
        output_list.append(lists[item]) 
      
    # Will add the values before 
    # n to the end of new list     
    for item in range(0, len(lists) - num):  
        output_list.append(lists[item]) 
          
    return output_list 
  
# Driver Code 
rotate_num = 1
list_1 = [1, 2, 3, 4, 5, 6] 
  
print(rightRotate(a, rotate_num)) 

[9, 0, 1, 2, 3, 4, 5, 6, 7, 8]


In [112]:
a

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)